In [1]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        #Initialize the decision tree with optional maximum depth limit
        self.max_depth = max_depth  # The maximum depth the tree can grow
        
    def fit(self, X, y):
        #Build the decision tree from training data (X, y)
        self.n_classes_ = len(np.unique(y))  # Number of unique classes in target
        self.n_features_ = X.shape[1]        # Number of features in dataset
        self.tree_ = self._grow_tree(X, y)  # Grow the tree starting from root
        
    def predict(self, X):
        #Predict class for input samples X
        return [self._predict(inputs) for inputs in X]  # Predict each sample
        
    def _gini(self, y):
        #Calculate Gini impurity for a set of target values
        # Count occurrences of each class
        _, counts = np.unique(y, return_counts=True)
        # Gini = 1 - sum(p_i^2) where p_i is proportion of class i
        impurity = 1 - np.sum([(count / len(y)) ** 2 for count in counts])
        return impurity
        
    def _best_split(self, X, y):
        #Finds the best split for a node (best feature and threshold)
        m = y.size  # Number of samples
        if m <= 1:
            return None, None  # Can't split if 1 or fewer samples
            
        # Count of each class in current node
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        # Calculate Gini impurity for current node
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None  # Initialize best split
        
        # Check all features to find best split
        for idx in range(self.n_features_):
            # Sort data along current feature
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            
            # Initialize counters for left and right nodes
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()  # Start with all in right, move to left
            
            # Try all possible split positions
            for i in range(1, m):
                c = classes[i - 1]  # Class of previous sample
                num_left[c] += 1    # Move to left
                num_right[c] -= 1    # Remove from right
                
                # Calculate Gini for both children
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )
                
                # Weighted average of children's Gini
                gini = (i * gini_left + (m - i) * gini_right) / m
                
                # Skip if same threshold as previous
                if thresholds[i] == thresholds[i - 1]:
                    continue
                    
                # Update best split if this one is better
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2  # Midpoint
        
        return best_idx, best_thr
        
    def _grow_tree(self, X, y, depth=0):
        #Recursively grow the decision tree
        # Count samples per class in current node
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        # The predicted class is the one with most samples
        predicted_class = np.argmax(num_samples_per_class)
        
        # Create new node (either leaf or decision node)
        node = Node(predicted_class=predicted_class)
        
        # Stop growing if max depth reached (unless None)
        if depth < self.max_depth if self.max_depth is not None else True:
            # Find best split
            idx, thr = self._best_split(X, y)
            if idx is not None:
                # Split data
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                
                # Set node properties
                node.feature_index = idx
                node.threshold = thr
                # Recursively grow left and right children
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node
        
    def _predict(self, inputs):
        #Predict class for a single sample by traversing the tree
        node = self.tree_  # Start at root
        while node.left:   # While not a leaf node
            # Go left or right based on feature threshold
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        # Return leaf node's prediction
        return node.predicted_class
    
class Node:
    #A node in the decision tree, either decision node or leaf node
    def __init__(self, *, predicted_class):
        self.predicted_class = predicted_class  # Class predicted at this node
        self.feature_index = 0      # Which feature to split on (for decision nodes)
        self.threshold = 0.0        # Threshold value for splitting
        self.left = None            # Left child (samples < threshold)
        self.right = None           # Right child (samples >= threshold)

    def is_leaf_node(self):
        #Check if this node is a leaf (has no children)
        return self.left is None and self.right is None

In [4]:
from sklearn.datasets import load_iris  # To load the iris flower dataset
from sklearn.model_selection import train_test_split  # To split data into train/test sets
from sklearn.metrics import accuracy_score  # To calculate prediction accuracy

iris = load_iris()
X = iris.data  # The feature measurements (our input data)
y = iris.target  # The species labels (what we want to predict)

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=42)

tree = DecisionTree(max_depth=3)  # Create decision tree with max depth 3
tree.fit(X_train, y_train)  # Train the tree using our training data

y_pred = tree.predict(X_test)  # Predict species for our test flowers


accuracy = accuracy_score(y_test, y_pred)  # Percentage of correct predictions

print(f"Accuracy: {accuracy}")  

Accuracy: 1.0
